# Name : Tonga junior cedric


In [48]:
# importation of necessary librairies
import random
from tabulate import tabulate
from collections import Counter
import math


In this part, it is a class that implements the whole mincount. I passed the W and d as parameters, and from that we can find the epsilon and delta. But you could also pass the epsilon and lambda as parameters and find the w and d. I think it doesn't bother.

I implemented the different functions and used the following formula to evaluate the difference between the predicted frequency and the actual frequency.

\begin{equation}
\mathrm{loss} = \frac{1}{n} \sum_{x \in \mathrm{universe}} \lvert \mathrm{actual}(x) - \mathrm{query}(x) \rvert
\end{equation}

where $n$ is the total number of inserted elements, $\mathrm{universe}$ is the set of all possible data elements, $\mathrm{actual}(x)$ is the actual frequency of data element $x$, and $\mathrm{query}(x)$ is the predicted frequency of data element $x$

In [49]:
class CountMin:
    def __init__(self, d, w, universe=['A','B','C','D','E','F','G']):

        self.d = d # depth of the table == number of hash functions
        self.w = w # width of the table
        self.universe = universe # the set of all possible items

        self.p = 1223543677 # a  large prime
        self.epsilon = math.e/self.w # base on w=math.e/epsilon
        self.delta = 1/math.exp(self.d) # based on d=math.log(1/delta)

        self.a = [random.randrange(self.p) for i in range(d)]  # random coefficients to have several hash functions
        self.b = [random.randrange(self.p) for i in range(d)]

        self.table_init() # initialize the table with zeros

    def table_init(self):
            
        '''
            we initialize the table with zeros
            we filled all the cells with zeros 
            
        '''
        self.table = [[0] * self.w for _ in range(self.d)]

    def hash(self, x,a,b):
        ''' 
            x: item
            hash function
            a: random coefficient
            b: random coefficient
            return: hash value of x with restriction in [0,w-1]
        '''
        return  ((a*hash(x)+b)%self.p)%self.w

    def get_hash(self, x):
        '''
            x: item
            return: list of hash values of x
        '''
        return [self.hash(x, self.a[i], self.b[i]) for i in range(self.d)]
    
    def print_table(self):
        '''
            print the table 

        '''
        
        print(tabulate([[f'h{(d)}']+self.table[d] for d in range(self.d)], tablefmt='grid'))
    
    def update(self, x): 
        
        '''
            x: item
            update the table if and only if x is in the universe
            so add 1 to the cell corresponding to the hash value of x
        '''
        if x in self.universe:
            for i in range(self.d):
                self.table[i][self.get_hash(x)[i]] += 1
        else:
            print('x is not in the universe')
    
    def query(self, x): 
        '''
            x: item
            return: the estimated frequency of x if x is in the universe
        '''
        if x in self.universe:
            
            return min([self.table[i][self.get_hash(x)[i]] for i in range(self.d)])
        else:
            return 'x is not in the universe'

    def random_insert(self, n): 
        '''
            n: number of items to be inserted
            insert n items randomly
            we update the table with the frequency of each item
            we return the frequency of each item
        '''
        self.table_init()
        choices = random.choices(self.universe, k=n)
        for x in choices:
            self.update(x)
        return Counter(choices)
    
    def evaluate(self, n): 
        '''
            n: number of items to be inserted
            return: loss between the actual frequency and the estimated frequency


            for that we define a loss function,which is the sum of the absolute difference between the actual frequency(real frequency)
            and the estimated frequency divided by n
            so if the loss is 0, it means that the estimated frequency is the same as the actual frequency(real frequency)
            and if the loss is different from 0, it means that the estimated frequency is different from the actual frequency

            finally,we print the epsilon, delta associated with the loss

        '''
        self.table_init()
        actual = self.random_insert(n)
        diff = list()
        for x in self.universe:
            diff.append(abs(actual[x] - self.query(x)))
            
            loss= sum(diff)/n
        print(f'for (ϵ={self.epsilon}, δ={self.delta}) approximation, the loss is {loss}')


Some examples

In [50]:
# initialize the counter with d=5 and w=5
counter2 = CountMin(5, 5)

# insert 1000 items randomly and evaluate the loss
counter2.evaluate(1000)

for (ϵ=0.543656365691809, δ=0.006737946999085467) approximation, the loss is 0.261


In [51]:
# initialize the counter with d=16 and w=4, this case seems like the approximative frequency is the same as the actual frequency
#but the  loss can be different due to random choices
counter1=CountMin(16,4)
# insert 1000 items randomly and evaluate the loss
counter1.evaluate(1000)

for (ϵ=0.6795704571147613, δ=1.1253517471925912e-07) approximation, the loss is 0.0


In [52]:
# initialize the counter with d=2 and w=4
counter3=CountMin(2,4)
# insert 1000 items randomly and evaluate the loss
counter3.evaluate(1000)

for (ϵ=0.6795704571147613, δ=0.1353352832366127) approximation, the loss is 0.975


# OBSERVATION : 

We can see that when the number of hash functions is small, there can be more collisions between the hash functions, and the approximation is larger than the actual frequency (because the predicted frequency cannot be negative, either it is equal to the actual one or it is larger). On the other hand, when the number of hash functions is high, the approximation of the frequency is better because there are fewer collisions.
We can therefore deduce that for a high number of hash functions, the approximate frequency is equal or tends to be equal to the real frequency, this is visible by the loss which is equal to 0 or tends more towards 0.
And if the loss moves further and further away from 0, it means that the approximate frequency is higher than the real frequency. So there are elements for which the approximate frequency is very different from the real one.

In [53]:
counter= CountMin(10, 30)  # 10 hash functions with arrays of size 30

print("with no update")
for x in ['A','B','C']:
    print(f'{x} : {counter.query(x)}')

with no update
A : 0
B : 0
C : 0


In [54]:
# try to update the table with the frequency of each item

# we can see that querie and update work well

for x in ['A','A','B','C']:
    counter.update(x)

print("with updating")
for x in ['A','B','C']:
    print(f'{x} : {counter.query(x)}')

with updating
A : 2
B : 1
C : 1


In [55]:
# print the table in other to see the frequency of each item

counter.print_table()

+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| h0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| h1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 2 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| h2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| h3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 3 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 

In [56]:
# random insert 1000 items of the universe
counter.random_insert(1000)

Counter({'D': 139, 'C': 144, 'E': 122, 'F': 144, 'B': 153, 'A': 142, 'G': 156})

In [57]:
# print the frequency of each item 
for x in counter.universe:
    print(f'{x} : {counter.query(x)}')

A : 142
B : 153
C : 144
D : 139
E : 122
F : 144
G : 156


In [58]:
# update the table with the item 'B'
counter.update('B')

In [59]:
# get frequency of 'B'
counter.query('B')

154